In [ ]:
from kafka import KafkaConsumer
from time import sleep
from json import dumps, loads
import json
from s3fs import S3FileSystem

In [ ]:
consumer = KafkaConsumer(
    'demotest',
    bootstrap_servers=['localhost:9092'],  # add your IP here
    value_deserializer=lambda x: loads(x.decode('utf-8')))

In [ ]:
for c in consumer:
    print(c.value)

In [ ]:
pip install mysql.connector

In [ ]:
import mysql.connector
from kafka import KafkaConsumer
import json

# Define your MySQL database credentials
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "stockx_db"
}

# Create a connection to the MySQL database
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

# Create a Kafka consumer
consumer = KafkaConsumer(
    'demotest',
    bootstrap_servers=['localhost:9092'],
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

for count, message in enumerate(consumer):
    data = message.value

    # Insert data into the MySQL table
    insert_query = "INSERT INTO StockData (`Index`, `Date`, `Open`, `High`, `Low`, `Close`, `Adj_Close`, `Volume`, `CloseUSD`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    values = (data['Index'], data['Date'], data['Open'], data['High'], data['Low'],
              data['Close'], data['Adj Close'], data['Volume'], data['CloseUSD'])
    cursor.execute(insert_query, values)
    connection.commit()

cursor.close()
connection.close()